In [1]:
%pip install datasets
%pip install pandas
%pip install argostranslate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 668.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 791.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.5/386.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart t

  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=1f0e573f72dc340262f2ab9a4f542a919d527587376940b2482d39d3f63f2bbd
  Stored in directory: /Users/marufahmed/Library/Caches/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset

dataset = load_dataset("Open-Orca/OpenOrca")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
from datasets import load_dataset
import pandas as pd


# Convert to Pandas DataFrame
df = pd.DataFrame(dataset['train'])  # Assuming 'train' split, adjust if needed

# Save the DataFrame to CSV
df.to_csv('/Users/marufahmed/Work/Machine Learning/Datasets/OpenOrca.csv', index=False, escapechar='\\')


In [ ]:
import argostranslate.package
import argostranslate.translate

from_code = "en"
to_code = "bn"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
available_package = list(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)[0]
download_path = available_package.download()
argostranslate.package.install_from_path(download_path)

# Translate
installed_languages = argostranslate.translate.get_installed_languages()
from_lang = list(filter(
        lambda x: x.code == from_code,
        installed_languages))[0]
to_lang = list(filter(
        lambda x: x.code == to_code,
        installed_languages))[0]
translation = from_lang.get_translation(to_lang)
translatedText = translation.translate("Hello World!")
print(translatedText)
# '¡Hola Mundo!'

: 

In [1]:
import pandas as pd
import argostranslate.package
import argostranslate.translate
from datasets import load_dataset
import time

# Load the original dataset
dataset = load_dataset("Open-Orca/OpenOrca")
df = pd.DataFrame(dataset['train'])

# Create a new DataFrame with only 'question' and 'response'
new_df = df[['question', 'response']].copy()

# Set up Argos Translate
from_code = "en"
to_code = "bn"
argostranslate.package.update_package_index()

# Check if the translation package is available for the specified languages
available_packages = argostranslate.package.get_available_packages()
matching_packages = list(filter(lambda x: x.from_code == from_code and x.to_code == to_code, available_packages))

if not matching_packages:
    raise ValueError("Translation package not available for the specified languages.")

available_package = matching_packages[0]
download_path = available_package.download()
argostranslate.package.install_from_path(download_path)

# Translate function
def translate_text(text):
    from_lang = list(filter(lambda x: x.code == from_code, argostranslate.translate.get_installed_languages()))[0]
    to_lang = list(filter(lambda x: x.code == to_code, argostranslate.translate.get_installed_languages()))[0]
    translation = from_lang.get_translation(to_lang)
    translated_text = translation.translate(text)
    return translated_text

# Translation loop
translated_pairs = 0
interval = 30  # In seconds
start_time = time.time()

for index, row in new_df.iterrows():
    question = row['question']
    response = row['response']

    try:
        # Translate question and response
        translated_question = translate_text(question)
        translated_response = translate_text(response)

        # Update the DataFrame with translated text
        new_df.at[index, 'question'] = translated_question
        new_df.at[index, 'response'] = translated_response

        translated_pairs += 1

        # Check if 30 minutes have passed and save progress
        if time.time() - start_time > interval:
            new_df.to_csv('translated_data.csv', index=False, escapechar='\\')
            print(f"Translated pairs: {translated_pairs}")
            start_time = time.time()

    except Exception as e:
        print(f"Error translating pair at index {index}: {e}")

# Save the final DataFrame
new_df.to_csv('translated_data_final.csv', index=False)
print("Translation completed. Total translated pairs:", translated_pairs)


KeyboardInterrupt: 

In [3]:
# import argostranslate

# from_code = "en"
# to_code = "bn"

# try:
#     available_packages = argostranslate.get_installed_languages()
#     if from_code in available_packages and to_code in available_packages:
#         print(f"Translation available from {from_code} to {to_code}")
#     else:
#         print(f"No translation available from {from_code} to {to_code}")
# except Exception as e:
#     print(f"Error: {e}")


Error: module 'argostranslate' has no attribute 'get_installed_languages'


In [6]:
from huggingface_hub import login

login()

In [ ]:
import pandas as pd
import requests
from datasets import load_dataset
import time

# Load the original dataset
dataset = load_dataset("Open-Orca/OpenOrca")
df = pd.DataFrame(dataset['train'])

# Create a new DataFrame with only 'question' and 'response'
new_df = df[['question', 'response']].copy()

# Set up LibreTranslate API endpoint
libretranslate_url = "http://localhost:5000/translate"  # Update with your LibreTranslate API endpoint

# Translate function using LibreTranslate API
def translate_text(text):
    try:
        response = requests.post(libretranslate_url, json={
            "q": text,
            "source": "en",
            "target": "bn"
        })
        response.raise_for_status()
        translated_text = response.json()["translatedText"]
        return translated_text
    except Exception as e:
        raise ValueError(f"Error translating text: {e}")

# Translation loop
translated_pairs = 0
interval = 30  # In seconds
start_time = time.time()

for index, row in new_df.iterrows():
    question = row['question']
    response = row['response']

    try:
        # Translate question and response
        translated_question = translate_text(question)
        translated_response = translate_text(response)

        # Update the DataFrame with translated text
        new_df.at[index, 'question'] = translated_question
        new_df.at[index, 'response'] = translated_response

        translated_pairs += 1

        # Check if 30 minutes have passed and save progress
        if time.time() - start_time > interval:
            new_df.to_csv('translated_data.csv', index=False, escapechar='\\')
            print(f"Translated pairs: {translated_pairs}")
            start_time = time.time()

    except Exception as e:
        print(f"Error translating pair at index {index}: {e}")

    # Print progress
    print(f"Processed pair {index + 1}/{len(new_df)}")

# Save the final DataFrame
new_df.to_csv('translated_data_final.csv', index=False)
print("Translation completed. Total translated pairs:", translated_pairs)
